# 모델 2 - 기능 3에 대해 추가 처리 가능한 모델2 작성 (1)
#### 모델 2는 모델 1에서 기능 3 지하철 트리거 인식 시 작동되는 모델임
#### 모델 1에서 기능 3으로 분류(카테고리 :3) 인 경우 해당 모델에서 역별 분류 작업 수행
#### 최종 수정일 : 20-05-27
* * *

# .npz 생성

* * *
## 1. 데이터 전처리
### 1.1 wav 파일 로딩 및 피처 생성

> [변경사항]
- 피처를 최대한 많이 생성하여 데이터를 가공함 -> drop out 효과 극대화 및 과적합 방지, 정확도 향상
- 이전 기능 별 분리되었던 모델(ver 2.*) 을 통합함
- 각 데이터 별 193 피처 추출
- row 통일 안함 (3~4sec)
- 원핫인코딩 안함
- 라벨을 1차원 배열로 변경 -> 카테고리 별 int값 출력하게 함
* * *
> [기존과 동일]
- 기능 1,2에 대해 사용 데이터는 뉴욕대학교  MARL의 URBANSOUND8K DATASET 일부와 일상 생활에서 녹음한 녹음 파일(.wav)를 활용 (2,622개, 1.96G)
- 기능 3에 대해 환승역 알림음을 트리거로 적용 (894개, 0.71G)

In [1]:
import numpy as np
import pandas as pd 

In [2]:
#wav 파일들의 피처 생성
#librosa 사용
#사용 특성은 mfcc, chroma_stft, melspectorgram, spectral_contrast, tonnetz로 총193
#딥러닝 모델만 사용할 예정 -> 피처 축소 생략
import glob
import librosa

In [3]:
# 오디오 불러오기 + 피쳐 생성
# 피쳐 193개
# row 통일 안시킴
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

In [4]:
#데이터 가공
#행렬로 변환
def parse_audio_files(filenames):
    rows = len(filenames)
    # feature는 각 파일 별 row(window) * 피처 의 2차원 행렬
    # labels은 파일 별 카테고리 int 값
    features, labels = np.zeros((rows,193)), np.zeros((rows, 1))
    i = 0
    for fn in filenames:
        try:
            mfccs, chroma, mel, contrast,tonnetz = extract_feature(fn)
            ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
            y_col = int(fn.split('-')[0])
        except:
            print("error : "+fn)
        else:
            features[i] = ext_features
            labels[i] = y_col
            print(y_col)
            i += 1
    return features, labels

In [5]:
audio_files = []
#0 : 사이렌
#1 : 자동차가 다가오는 소리(엔진소리)
#2 : 자동차 경적소리
#4 : 환승역 안내음
audio_files.extend(glob.glob('*.wav'))

In [6]:
print(len(audio_files))

129


In [7]:
files = audio_files
X, y= parse_audio_files(files)

0
0
0
1
1
1
10
10
10
11
11
11
12
12
12
13
13
13
14
14
14
15
15
15
16
16
16
17
17
17
18
18
18
19
19
19
2
2
2
20
20
20
21
21
21
22
22
22
23
23
23
24
24
24
25
25
25
26
26
26
27
27
27
28
28
28
29
29
29
3
3
3
30
30
30
31
31
31
32
32
32
33
33
33
34
34
34
35
35
35
36
36
36
37
37
37
38
38
38
39
39
39
4
4
4
40
40
40
41
41
41
42
42
42
5
5
5
6
6
6
7
7
7
8
8
8
9
9
9


In [8]:
#.npz
np.savez('data', X=X, y=y)